In [1]:
import tensorflow as tf

# Computational graphs 

## Node takes a input and produces a output 

### Types of nodes
* **Constant:**
    Has no input, but always produces a output

In [3]:
node1 = tf.constant(3.0, dtype=tf.float64)

In [4]:
print(node1)

Tensor("Const:0", shape=(), dtype=float64)


Note that output is a null, we have to actually run the graph for the node to actually take a value.

In [10]:
sess = tf.Session()
print(sess.run(node1))

3.0


As expected, the node takes on the value !

* **Operator Nodes:** Addition, Subtraction.. etc

In [11]:
node2 = tf.constant(4.0, dtype=tf.float64) 

In [14]:
node3 = tf.add(node1, node2) # Node 3 for addition

In [15]:
sess.run(node3)

7.0

This is nothing magical, the true magic of tf is in its ability to pipe operations through this graph.

Consider the following example,

We create a graph that takes in 2 *values* and returns their sum.

In [16]:
a = tf.placeholder(tf.float64)
b = tf.placeholder(tf.float64)
adder_node = a + b

In [17]:
sess.run(adder_node, {a: 3.0, b: 4.5})

7.5

**Now the question is how is this different from a normal function one would write in numpy ?**


In [20]:
sess.run(adder_node, {a:[2, 3, 5.0], b:[10, 121, -10.0]})

array([  12.,  124.,   -5.])

i.e you are not limited by data structure

* **Variable**: Constants that can be modified, specifically weights and bias in the networks 

In [78]:
W = tf.Variable([0.1, 0.2, 0.3], dtype=tf.float32)
c = tf.Variable([0.1], dtype=tf.float32)
x = tf.placeholder(tf.float32)

In [79]:
linear_model = tf.tensordot(W, x, axes=1) + c

In [80]:
init = tf.global_variables_initializer()
sess.run(init)

Unlike constants, variables have to initalized by the before they can be used in a operation.

In [81]:
sess.run(linear_model, {x:[1,2,3]})

array([ 1.50000012], dtype=float32)

Further these graphs can be interveaved to represent complex operations

In [82]:
y = tf.placeholder(tf.float32)
squared_delta = tf.square(linear_model - y)
loss = tf.reduce_sum(squared_delta)

In [83]:
sess.run(loss, {x:[1,2,3], y:[1.5]})

1.4210855e-14

The values of the variables can be reassigned on the go...

In [84]:
sess.run([tf.assign(W, [1,2,3]), tf.assign(b, [0])])

[array([ 1.,  2.,  3.], dtype=float32), array([ 0.], dtype=float32)]

In [86]:
sess.run(loss, {x:[1,1,1], y:[6]})

0.0099999812

# Magic !!

**tf.gradiets** does the backprop for us, based on a model of the graph !

In [87]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

In [89]:
sess.run(init) # reset values to incorrect defaults.
for i in range(1000):
  sess.run(train, {x: [1, 2, 3], y: [0]})

print(sess.run([W, b]))

[array([ -4.43082282e-09,  -8.86164564e-09,   8.86164742e-09], dtype=float32), array([ 0.30000001], dtype=float32)]


**tf.estimator** provides a high level abstraction for *training, evalutating and managing data sets*

Not only we are limited to the existing models, but new models can be built into it.

In [90]:
import numpy as np
import tensorflow as tf

# Declare list of features, we only have one real-valued feature
def model_fn(features, labels, mode):
  # Build a linear model and predict values
  W = tf.get_variable("W", [1], dtype=tf.float64)
  b = tf.get_variable("b", [1], dtype=tf.float64)
  y = W*features['x'] + b
  # Loss sub-graph
  loss = tf.reduce_sum(tf.square(y - labels))
  # Training sub-graph
  global_step = tf.train.get_global_step()
  optimizer = tf.train.GradientDescentOptimizer(0.01)
  train = tf.group(optimizer.minimize(loss),
                   tf.assign_add(global_step, 1))
  # EstimatorSpec connects subgraphs we built to the
  # appropriate functionality.
  return tf.estimator.EstimatorSpec(
      mode=mode,
      predictions=y,
      loss=loss,
      train_op=train)

estimator = tf.estimator.Estimator(model_fn=model_fn)
# define our data sets
x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7., 0.])
input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=None, shuffle=True)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=1000, shuffle=False)
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_eval}, y_eval, batch_size=4, num_epochs=1000, shuffle=False)

# train
estimator.train(input_fn=input_fn, steps=1000)
# Here we evaluate how well our model did.
train_metrics = estimator.evaluate(input_fn=train_input_fn)
eval_metrics = estimator.evaluate(input_fn=eval_input_fn)
print("train metrics: %r"% train_metrics)
print("eval metrics: %r"% eval_metrics)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/7t/gd48cfs92fbdl56n68skvq3r0000gn/T/tmpsed4ob0s', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/7t/gd48cfs92fbdl56n68skvq3r0000gn/T/tmpsed4ob0s/model.ckpt.
INFO:tensorflow:loss = 6.2835420278, step = 1
INFO:tensorflow:global_step/sec: 628.117
INFO:tensorflow:loss = 0.0502837995482, step = 101 (0.164 sec)
INFO:tensorflow:global_step/sec: 540.672
INFO:tensorflow:loss = 0.00451182418842, step = 201 (0.185 sec)
INFO:tensorflow:global_step/sec: 422.474
INFO:tensorflow:loss = 0.000274428900752, step = 301 (0.233 sec)
INFO:tensorflow:global_step/sec: 554.496
INFO:tensorflow:loss = 2.25488926414e-05, step = 40